<a href="https://colab.research.google.com/github/Yasmiiiiine/12-Month-Forecast-With-LSTM/blob/master/TimeSeries_for_sum_sales_Generic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install**

In [11]:
!pip install chart_studio

# **Imports**

In [0]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

import numpy as np
import pandas as pd

# **Functions**

**Period to date**

In [0]:
def periode_to_date(x):
  return x[:4]+'-'+x[4:6]+'-01'

**Predict Sale**

In [0]:
def pred_sales(x, model):
  x1 = x['prev_sales']
  x2 = x['SUM_VENTE']
  x_input = array([x1, x2]).reshape((1, n_input, n_features))
  return model.predict(x_input, verbose=0)[0][0]

**Read data**

In [0]:
def read_data(filename, entite, offre, periode_limite):

  df_sales = pd.read_csv(filename, sep=';') #'VENTES_SBD.csv'
  df_sales['date'] = pd.to_datetime(df_sales['PERIODE'].apply(lambda x : periode_to_date(str(x))))

  #represent month in date field as its first day
  df_sales['date'] = df_sales['date'].dt.year.astype('str') + '-' + df_sales['date'].dt.month.astype('str') + '-01'
  df_sales['date'] = pd.to_datetime(df_sales['date'])

  df = df_sales
  df_sales = df_sales[(df_sales['LIBELLE_OFFRE']==offre)]
  df_sales = df_sales[(df_sales['ENTITE']==entite)]
  df_sales = df_sales[(df_sales['PERIODE'] <= periode_limite)]

  #create a new dataframe to model the difference
  df_diff = df_sales.copy()
  #add previous sales to the next row
  df_diff['prev_sales'] = df_diff['SUM_VENTE'].shift(1)
  #drop the null values and calculate the difference
  df_diff = df_diff.dropna()

  return df_diff

In [0]:
def create_generator(df_diff, val_rate):
  
  n_input = 2
  n_features = 1
  val_limit = int(len(df_diff.index)*val_rate)

  series = array(df_diff['SUM_VENTE'][:val_limit])
  series = series.reshape((len(series), n_features))

  series_val = array(df_diff['SUM_VENTE'][val_limit:])
  series_val = series_val.reshape((len(series_val), n_features)) 
    
  # define generator
  generator = TimeseriesGenerator(series, series, length=n_input, batch_size=6)
  val_generator = TimeseriesGenerator(series_val, series_val, length=n_input, batch_size=6)

  return generator, val_generator

**Create model**

In [0]:
def create_model():

  n_input = 2
  n_features = 1

  # define model
  model = Sequential()
  model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')
  
  return model

**Plot**

In [0]:
def get_fig(df_diff):
  fig = go.Figure(data=[
                      go.Scatter(x=df_diff['date'], y=df_diff['SUM_VENTE'], name='actual'), 
                      go.Scatter(x=df_diff['date'], y=df_diff['pred'], name='predicted')
                      ],
    layout=go.Layout(title='Montly Sales forecasting')
    )
  return fig

# **Experiments**

In [0]:
data = read_data('VENTES_SBD.csv', 'SBD', 'DATA MOBILE', 202003)

In [20]:
data.head()

,ENTITE,ANNEE,PERIODE,LIBELLE_OFFRE,VENTE_MOIS,SUM_VENTE,date,prev_sales
1,SBD,2017,201702,DATA MOBILE,503.0,769.0,2017-02-01,266.0
2,SBD,2017,201703,DATA MOBILE,805.0,1574.0,2017-03-01,769.0
3,SBD,2017,201704,DATA MOBILE,286.0,1860.0,2017-04-01,1574.0
4,SBD,2017,201705,DATA MOBILE,648.0,2508.0,2017-05-01,1860.0
5,SBD,2017,201706,DATA MOBILE,253.0,2761.0,2017-06-01,2508.0


In [0]:
gen, gen_val = create_generator(data, 0.3)